In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns

In [ ]:
# Cargamos los datos y seleccionamos las columnas
df = pd.read_parquet("../Preprocessing/flightsCleaned.parquet")
df.head()

In [ ]:
# Seleccionamos las variables que nos interesan para este análisis
df = df.loc[:, ['AIRLINE', 'FLIGHT_NUMBER','ARRIVAL_DELAY','DISTANCE']]

In [ ]:
# Distribución de los vuelos en función de la aerolínea
airlines = pd.DataFrame(df.groupby(df['AIRLINE'])["FLIGHT_NUMBER"].count())
airlines = airlines.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
airlines["ON_TIME_FLIGHTS"] =  df[df["ARRIVAL_DELAY"]<=0].groupby(df['AIRLINE'])["FLIGHT_NUMBER"].count()
airlines["DELAYED_FLIGHTS"] =  df[df["ARRIVAL_DELAY"]>0].groupby(df['AIRLINE'])["FLIGHT_NUMBER"].count()
airlines["ON_TIME_PERCENTAGE"] = np.round(airlines['ON_TIME_FLIGHTS']/airlines['TOTAL_FLIGHTS']*100,2)
airlines["DELAY_PERCENTAGE"] = np.round(airlines['DELAYED_FLIGHTS']/airlines['TOTAL_FLIGHTS']*100,2)
airlines = airlines.sort_values('TOTAL_FLIGHTS',ascending=False)
airlines = airlines.reset_index(level=0, drop=False)
airlines

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=airlines['AIRLINE'],
    y=airlines['TOTAL_FLIGHTS'],
    name='Total Flights',
    marker_color=px.colors.qualitative.Vivid[5]
))
fig.add_trace(go.Bar(
    x=airlines['AIRLINE'],
    y=airlines['DELAYED_FLIGHTS'],
    name='Delayed Flights',
    marker_color=px.colors.qualitative.Vivid[9]
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    title="Flights Distribution per Airline",
    xaxis_title="Airlines",
    yaxis_title="Air traffic",
    legend_title="Leyend",
    template="plotly_dark",
    barmode='group', 
    xaxis_tickangle=-45   
)

fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

Creamos una serie de variables relacionadas con las rutas y las multas

In [ ]:
# Creamos la columna tipo de vuelos según la distancia
condition_distance = [
    (df['DISTANCE'] < 750) ,
    (df['DISTANCE'] >= 750) & (df['DISTANCE'] <1500),
    (df['DISTANCE'] >= 1500)]

choice_distance = ['Short', 'Mid', 'Long']

# Creamos la columna tipo de retraso
condition_delay = [
    (df['ARRIVAL_DELAY'] <= 0) ,
    (df['ARRIVAL_DELAY'] > 0) & (df['ARRIVAL_DELAY'] <= 30),
    (df['ARRIVAL_DELAY'] > 30) & (df['ARRIVAL_DELAY'] <=60),
    (df['ARRIVAL_DELAY'] > 60)]

choice_delay = ['Early arrival', '(0-30)mins', '(30-60)mins','>1h']

# Añadimos las columnas
df['DISTANCE_TYPE'] = np.select(condition_distance, choice_distance, default='Not Specified')
df['DELAY_TYPE'] = np.select(condition_delay, choice_delay, default='Not Specified')

In [ ]:
# Agrupamos por aerolineas la info de los retrasos
level_count = pd.DataFrame(df.groupby("AIRLINE")["DELAY_TYPE"].value_counts()).rename(columns = {"DELAY_TYPE": "count"}).reset_index()

fig = px.histogram(level_count, x="AIRLINE", y="count",barnorm='percent', color="DELAY_TYPE",text_auto='.2f',
                   title="Flights Distribution per Airline", color_discrete_sequence=px.colors.qualitative.Vivid, template="plotly_dark")

fig.update_layout(
    title="Flights Distribution per Airline",
    xaxis_title="Airlines",
    yaxis_title="% of flights per delay type",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified"
    
)


fig.show()

El 80% de los vuelos de todas las aerolíneas están exentos de multa dado el tipo de retraso. No obstante, Frontier y Spririt destacan por ser las dos compañías con menor % de vuelos libres de multas. Vamos a analizar más detenidamente los vuelos objetivo de multa

In [ ]:
# Agrupamos por aerolineas la info de los retrasos
level_count = level_count.loc[(level_count['DELAY_TYPE'] == '(30-60)mins') | (level_count['DELAY_TYPE'] == ">1h")]

fig = px.histogram(level_count, x="AIRLINE", y="count", color="DELAY_TYPE",text_auto='.2f',
                   title="Flights Distribution per Airline", color_discrete_sequence=px.colors.qualitative.Vivid[2:], template="plotly_dark")

fig.update_layout(
    title="Late arrival flights distribution per Airline",
    xaxis_title="Airlines",
    yaxis_title="Amount of flights per delay type",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified"
    
)


fig.show()

Alarmante el caso de Southwest. En la gráfica anterior veíamos que en torno al 89% de sus vuelos llegaban a destino con un máximo de retraso de 30 min. En esta gráfica se observa que ese 11% restante incluye una gran cantidad de vuelos, lo que podría costarle a la compaía mucho dinero.

# FINES

Vamos a analizar cuánto le costaría a cada compañia dada esta distribución de retrasos

**RECAP:** Las compañias pagarán una multa en función de la distancia del trayecto y el tiempo de retraso. Además también deberén reembolsar a los pasajeros un % del importe de su billete como consecuencia del tiempo de retraso

Para calcular cuánto se deberá pagar en cada caso necesitamos hacer una serie de estimaciones las cuales detallaremos a continuación.

**IMPORTE DEL BILLETE**

"In 2015, an average flight cost $430." https://www.mercurynews.com/2016/07/08/see-how-the-cost-of-a-flight-has-changed-since-1963/

Asociamos avg flight con distancia media. También asumimos que a mayor distancia mayor precio (un 30% superior al avg), así mismo como a menor distancia, menor precio (un 30% inferior al avg). Como consecuencia, asumimos que el importe de cada billete en función de la distancia es el siguiente:

In [ ]:
AVG_TICKET_SHORT = 0.7*430 # $301
AVG_TICKET_MID = 430
AVG_TICKET_LONG = 1.3*430  # $559

**NIVEL DE OCUPACIÓN**

Por otro lado, necesitamos estimar el número de pasajeros por vuelo para estimar el importe a pagar a los pasajeros en los vuelos que sufren mayores retrasos. De acuerdo con https://datos.bancomundial.org/indicator/IS.AIR.PSGR?locations=US en el año 2015 se transportaron en Estados Unidos un total de 798,222,000 pasajeros en todo el año. Asumimos una distribución uniforme de pasajeros en los vuelos dado que no existen evidencias claras de que la capacidad de los aviones sea fija en función de la distancia.

Dicho esto:

In [ ]:
total_passsengers = 798222000
total_flights = len(df)
pass_per_flight = round(total_passsengers/total_flights,2) # 139,76
pass_per_flight = int(round(pass_per_flight,0))
pass_per_flight

#### Cálculo del importe a pagar 

In [ ]:
# Retrasos (30-60) mins --> multa en función de la distancia + 50% del importe del billete
AVG_TICKET_SHORT_TYPE_I =  5000 + 0.5*pass_per_flight*AVG_TICKET_SHORT
AVG_TICKET_MID_TYPE_I   = 10000 + 0.5*pass_per_flight*AVG_TICKET_MID
AVG_TICKET_LONG_TYPE_I  = 20000 + 0.5*pass_per_flight*AVG_TICKET_LONG

# Retrasos >1h --> multa en función de la distancia + 100% del importe del billete
AVG_TICKET_SHORT_TYPE_II =  7500 + 0.5*pass_per_flight*AVG_TICKET_SHORT
AVG_TICKET_MID_TYPE_II   = 20000 + 0.5*pass_per_flight*AVG_TICKET_MID
AVG_TICKET_LONG_TYPE_II  = 40000 + 0.5*pass_per_flight*AVG_TICKET_LONG

# Creamos una lista de los $ a pagar en función de la distancia y el retraso de los vuelos
choice_multa = [0,AVG_TICKET_SHORT_TYPE_I,AVG_TICKET_MID_TYPE_I,AVG_TICKET_LONG_TYPE_I,
                AVG_TICKET_SHORT_TYPE_II,AVG_TICKET_MID_TYPE_II,AVG_TICKET_LONG_TYPE_II]

In [ ]:
# Creamos la columna correspondiente a la multa
condition_multa = [
    (df['DELAY_TYPE'] == 'Early arrival') | (df['DELAY_TYPE'] == '(0-30)mins'),
    (df['DISTANCE_TYPE'] == 'Short') & (df['DELAY_TYPE'] == '(30-60)mins'),
    (df['DISTANCE_TYPE'] == 'Mid') & (df['DELAY_TYPE'] == '(30-60)mins'),
    (df['DISTANCE_TYPE'] == 'Long') & (df['DELAY_TYPE'] == '(30-60)mins'),   
    
    (df['DISTANCE_TYPE'] == 'Short') & (df['DELAY_TYPE'] == '>1h'),
    (df['DISTANCE_TYPE'] == 'Mid') & (df['DELAY_TYPE'] == '>1h'),
    (df['DISTANCE_TYPE'] == 'Long') & (df['DELAY_TYPE'] == '>1h')]

# Añadimos la columna de multas
df['FINE'] = np.select(condition_multa, choice_multa, default=int())

## Distribución de las multas por aerolínea y retraso en función de la distancia

In [ ]:
fines = pd.DataFrame(df.groupby(df['AIRLINE'])["FLIGHT_NUMBER"].count())
fines = fines.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
fines["AMOUNT_DUE"] = df.groupby(df['AIRLINE'])["FINE"].sum()
fines = fines.sort_values('AMOUNT_DUE',ascending=False)
fines = fines.reset_index(level=0, drop=False)
fines

In [ ]:
# Seleccionamos las 5 aerolíneas con mayor cantidad de $$ a pagar
top_5 = list(fines['AIRLINE'].loc[0:4])
top_5_fined = df[df.AIRLINE.isin(top_5)]
top_5_fined.head()

In [ ]:
data_sunb = pd.DataFrame(top_5_fined.groupby(df['AIRLINE'])["FLIGHT_NUMBER"].count())
data_sunb = data_sunb.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
data_sunb["AMOUNT_SHORT"] =  top_5_fined[top_5_fined["DISTANCE_TYPE"]== "Short"].groupby(df['AIRLINE'])["FINE"].sum()
data_sunb["AMOUNT_MID"] =  top_5_fined[top_5_fined["DISTANCE_TYPE"]== "Mid"].groupby(df['AIRLINE'])["FINE"].sum()
data_sunb["AMOUNT_LONG"] =  top_5_fined[top_5_fined["DISTANCE_TYPE"]== "Long"].groupby(df['AIRLINE'])["FINE"].sum()
data_sunb = data_sunb.reset_index(level=0, drop=False)
data_sunb

In [ ]:
total = data_sunb['TOTAL_FLIGHTS'].sum()
values =[total]
values

In [ ]:
top5 = list(data_sunb['AIRLINE'])
values = [total]+list(data_sunb.iloc[0,1:])+list(data_sunb.iloc[1,1:])+list(data_sunb.iloc[2,1:])+list(data_sunb.iloc[3,1:])+list(data_sunb.iloc[4,1:])
len(values)

fig =go.Figure(go.Sunburst(
    
#         ids=["North America", "Europe", "Australia", "North America - Football", "Soccer",
#             "North America - Rugby", "Europe - Football", "Rugby",
#             "Europe - American Football","Australia - Football", "Association",
#             "Australian Rules", "Autstralia - American Football", "Australia - Rugby",
#             "Rugby League", "Rugby Union"],
    
        labels=["Airlines", top5[0], "Short 1","Mid 1","Long 1",
                top5[1],"Short 2","Mid 2","Long 2",
                top5[2],"Short 3","Mid 3","Long 3",
                top5[3],"Short 4","Mid 4","Long 4",
                top5[4],"Short 5","Mid 5","Long 5"],
        
        parents=["","Airlines",top5[0],top5[0],top5[0],
                    "Airlines",top5[1],top5[1],top5[1],
                    "Airlines",top5[2],top5[2],top5[2],
                    "Airlines",top5[3],top5[3],top5[3],
                    "Airlines",top5[4],top5[4],top5[4]],
                           
#         values=[100, 140, 120, 100, 200, 
#                 600,600,400,400,
#                 200,200,100,500,
#                 600,600,700,500,
#                 600,600,500,400],
        values = values,
        #marker_colors = px.colors.qualitative.Vivid,
))


fig.update_layout()

fig.update_layout(
    title="Fines distribution per airline and type of flight",
    template="plotly_dark",
    margin = dict(t=60, l=0, r=0, b=30),
    #hovermode="x unified"
    
)


fig.show()

# Cambiar número de vuelos por $?

# Airtime - distance